## MSTICPy and Notebooks in InfoSec

---

<h1 style="border: solid; padding:5pt; color:black; background-color:#909090">Session 2 - MSTICPy Configuration</h1>

---

## What this session covers:

- Viewing current settings
- Configuration tools
- Add a Sentinel Workspace
- Add a Kusto Cluster
- Configure KeyVault settings
- Add a Threat Intel Provider
- Add a GeoIP Provider
- Verifying Azure configuration


## Prerequisites
- Python >= 3.8 Environment
- Jupyter installed
- MSTICPy installed
- Azure CLI installed
- Run `az login`
- Account and API key for one or more TI providers
- Account and API key for MaxMind GeoLite


In [1]:
import msticpy as mp
mp.init_notebook()

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090"><i>msticpyconfig.yaml</i> Structure</a>

---

### [Reference: MSTICPy Package Configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
> Note document covers most but not all configuration options (it's difficult to keep this up-to-date).


## msticpyconfig.yaml

```yaml
Azure:
  auth_methods: [cli, msi, devicecode]
  cloud: global
DataProviders:
  # Miscellaneous Data providers
  Browshot:
    Args:
      # AuthKey: [PLACEHOLDER]
  # Previous Kusto cluster definition
  Kusto-MDE: 
    Args:
      # Cluster: https://wcdscrubbedservice.kusto.windows.net
      # IntegratedAuth: true
TIProviders:
  TorExitNodes:
    Primary: true
    Provider: Tor
  VirusTotal:
    Args:
      AuthKey:
        KeyVault: null
    Primary: true
    Provider: VirusTotal
OtherProviders:
  # GeoIP providers - should be in DataProviders!
  GeoIPLite:
    Args:
      AuthKey:
        KeyVault: null
      DBFolder: ~/.msticpy
    Provider: GeoLiteLookup
AzureSentinel:
  # Sentinel workspace configuration
  Workspaces:
    Workspace_Tag:
      # workspace ID, tenant, ext
    Workspace2_Tag:
      # ...
KustoClusters:
  # Kusto cluster definitions (for NEW Kusto driver)
  ClusterDefaults:
    Args:
      TenantId: 69d28fd7-42a5-48bc-a619-af56397b9f28
  Cluster1:
    Args:
      Cluster: https://uscluster.kusto.windows.net
  Cluster2:
    Args:
      Cluster: https://eucluster.kusto.windows.net
KeyVault:
  # Optional - KV tenant, sub, vault name

```

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Show current settings</a>

---

You can print out current settings from `msticpy.settings.get_config()`

> **Note**: internally we do some additional mappings so you might
> see things in here that are not in your config file.

In [2]:
mp.settings.get_config()

{'msticpy': {'FriendlyExceptions': True},
 'QueryDefinitions': {'Default': ['queries']},
 'Azure': {'auth_methods': ['cli', 'interactive'], 'cloud': 'global'},
 'AzureSentinel': {'Workspaces': {'ASIHuntOMSWorkspaceV4': {'ResourceGroup': 'asihuntomsworkspacerg',
    'SubscriptionId': '40dcc8bf-0478-4f3b-b275-ed0a94f2c013',
    'TenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
    'WorkspaceId': '52b1ab41-869e-4138-9e40-2a4457f09bf0',
    'WorkspaceName': 'ASIHuntOMSWorkspaceV4'},
   'CCIS': {'ResourceGroup': 'securityinsightsrg',
    'SubscriptionId': 'b5fe39b7-119a-4622-a921-236607a8d59a',
    'TenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
    'WorkspaceId': 'd2a20a39-c646-4783-a490-59899e3a6591',
    'WorkspaceName': 'Security-Insights'},
   'Centrica': {'ResourceGroup': 'azsu-rg-azure-sentinel',
    'SubscriptionId': '23d6e98b-d73d-476e-8381-1faf272fd0f4',
    'TenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
    'WorkspaceId': '1ab17267-66c0-473e-8a80-da21cc7a0828',
    'Wo

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Settings Tools - MpConfigFile and MpConfigEdit</a>

---

### [Reference: MSTICPy Settings Editor](https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html)

## MpConfigFile - settings utilities interactive and command line

In [3]:
config_file = mp.MpConfigFile()
config_file

In [4]:
config_file.view_settings()


### MpConfigEdit - interactive settings editor

If you haven't authenticated using Azure CLI, do that now (in a console window or the notebook) before running the `az_connect()` cell.

```bash
c:\users\ian > az login
/home/ian $ az login
```

You may need to go through a "device code" login:
- copy the code
- navigate to the login page given in the URL
- paste the code
- authenticate as normal

You can also do this in the notebook by uncommenting and running the following cell

In [ ]:
# !az login

In [8]:
# We'll need to authenticate to Azure to use Workspace details resolution
mp.az_connect()

AzCredentials(legacy=<msticpy.auth.cred_wrapper.CredentialWrapper object at 0x7fdbc2561000>, modern=<azure.identity._credentials.chained.ChainedTokenCredential object at 0x7fdbbe305660>)

## Example - Adding a Sentinel Workspace

We will: 

1. Add a new workspace with the following values:
```
    - WorkspaceID: 52b1ab41-869e-4138-9e40-2a4457f09bf0
    - TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
```
2. Resolve all settings for the workspace
3. DO NOT set it as the Default Workspace
4. Update and Save Settings
5. Verify Settings have been re-read

In [9]:
# WorkspaceID: 52b1ab41-869e-4138-9e40-2a4457f09bf0
# TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
mp.MpConfigEdit()

Label(value='Loading. Please wait.')

In [ ]:
mp.MpConfigFile().view_settings()

In [12]:
# test
sentinel_settings = mp.settings.get_config("AzureSentinel.Workspaces")
expected_keys = {"ResourceGroup", "SubscriptionId", "TenantId", "WorkspaceId", "WorkspaceName"}
assert "CyberSecuritySOC" in sentinel_settings
assert "Default" in sentinel_settings

assert all(key in expected_keys for key in sentinel_settings["CyberSecuritySOC"])
assert all(val is not None for val in sentinel_settings["CyberSecuritySOC"].values())

# If all is well you should see a green tick with no other output

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 1 - Add a Kusto Cluster</a>

We've recently updated the Kusto driver and it now uses a more logical layout
of cluster settings. 

<div style="border: solid; padding:5pt">The Config editor doesn't support this yet - so we're going to do this the old-fashioned way!</div>

1. Open the `msticpyconfig.yaml` in VSCode or editor of your choice
2. If the `KustoClusters` top-level key does not exist, add it.
3. Add a cluster `Kusto-Firecon23`:
4. Add an `Args` sub-key and then add the following values.
```
   - Cluster: https://msticpytraining.eastus.kusto.windows.net/
   - TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
   - IntegratedAuth: True
```
5. Save the file
6. Run the following cell to re-read the settings from disk


The format of a Kusto cluster entry that you need to add looks like this:
```yaml
KustoClusters:
   Kusto-Firecon23:
      Args:
         Cluster: https://msticpytraining.eastus.kusto.windows.net/
         IntegratedAuth: True
         TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      ...
```

`Kusto-Firecon23` is the cluster instance name (can be anything)<br>
`ClientId` and `ClientSecret` are not needed if `IntegratedAuth` is true

In [5]:
# tell msticpy to refresh its config from the file
mp.refresh_config()

In [ ]:
# test
kusto_settings = mp.settings.get_config("KustoClusters")
assert "Kusto-Firecon23" in kusto_settings

assert "Cluster" in kusto_settings["Kusto-Firecon23"]["Args"]
assert "TenantId" in kusto_settings["Kusto-Firecon23"]["Args"]
assert "IntegratedAuth" in kusto_settings["Kusto-Firecon23"]["Args"]


You should be able to authenticate to this cluster

In [19]:
qry_prov = mp.QueryProvider("Kusto_New")
qry_prov.connect(cluster="Kusto-Firecon23")

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 2 - KeyVault Settings</a>

Since we only need to read to and write from the KeyVault, we only need a subset of
the settings.
- Authority: global
- TenantId: <tenant-guid>
- UseKeyring: false
- VaultName: <name>

If we needed to create a vault from MSTICPy we would also need:
- AzureRegion, SubscriptionId, ResourceGroup

### Steps:
1. Use the Key Vault tab
2. Confirm or enter values for:
```
    - Authority: global
    - TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
    - UseKeyring: False (unchecked)
    - VaultName: msticpy-training
```
3. Optional settings:
```
    - AzureRegion: East US
    - SubscriptionId: 40dcc8bf-0478-4f3b-b275-ed0a94f2c013
    - ResourceGroup: MSTICPy
```
4. Update and save settings


In [7]:
mp.MpConfigEdit()

Label(value='Loading. Please wait.')

In [21]:
mp.MpConfigFile().view_settings()

In [23]:
# test
kv_settings = mp.settings.get_config("KeyVault")

assert kv_settings["Authority"] == "global"
assert kv_settings["TenantId"] == "72f988bf-86f1-41af-91ab-2d7cd011db47"
assert kv_settings["UseKeyring"] == False
assert kv_settings["VaultName"] == "msticpy-training"


## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 3 - Add Threat Intel. Providers</a>

Add configuration for a TI provider

1. Use the TI Providers tab
2. Pick a TI provider and add it
3. Fill in details (use the Text option) and Update
4. Save the settings

If you have API keys for other providers, add these.

You should have API keys for:
* VirusTotal
* AlientVault OTX
* IBM XForce\*

> \* Note XForce has both an API Key and and API Password<br>
> The API Key goes in the **ApiID** box, the password goes in the **AuthKey** box

Don't worry if you don't have accounts/keys for all of these providers - just one or two will be fine.

In [ ]:
mp.MpConfigEdit()

In [ ]:
mp.MpConfigFile().view_settings()

In [ ]:
# test
ti_settings = mp.settings.get_config("TIProviders")
assert "VirusTotal" in ti_settings

assert ti_settings["VirusTotal"]["Primary"]
assert ti_settings["VirusTotal"]["Provider"] == "VirusTotal"
assert "AuthKey" in ti_settings["VirusTotal"]["Args"]


Settings for TI Providers should look like this
```
'TIProviders': {'TorExitNodes': {'Primary': True, 'Provider': 'Tor'},
                 'VirusTotal': {'Args': {'AuthKey': '[YOUR VT API KEY]',
                                         'UseVT3PrivateAPI': False},
                                'Primary': True,
                                'Provider': 'VirusTotal'}}}
```

In [ ]:
# Test

mp.TILookup().lookup_iocs(["54.69.246.204"])


## Alternatives to storing keys in your config file
- You can use the **EnvironmentVar** option to store secrets in environment variables
- You can upload secrets to KeyVault and use them from there.
- You need to configure settings of your vault on the Key Vault tab to do this.
- You can use KeyRing on supported platforms without Key Vault backing (message me for details on this)

Default Key Vault secret names are built from the path of the value. E.g.:
```
    TIProviders-VirusTotal-Args-AuthKey
```

If you have a secret already in a Key Vault, uncheck `Def KV Path` and
type the name of your secret. 

You can also specify `Vaultname/Secretname` to 
us a secret from a named Vault rather than the default.

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 4 - Add a GeoIP Provider</a>

1. Use the GeoIP Providers tab
2. Add an entry for GeoIPLite
   - AuthKey: *your_auth_key*
   - DBFolder:  `./.msticpy`
3. Save settings
4. View settings
5. Test that the settings work


In [ ]:
# configure
mp.MpConfigEdit()

In [ ]:
mp.MpConfigFile().view_settings()

Settings for OtherProviders should look like this
```
'OtherProviders': {'GeoIPLite': {'Args': {'AuthKey': 'YOUR_GEO_LITE_KEY',
                                           'DBFolder': './.msticpy'},
                                  'Provider': 'GeoLiteLookup'}},
```

In [ ]:
# Test

IpAddress.geoloc("174.34.43.21")

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 5 - Confirm Azure Settings</a>

1. Use the Azure tab
2. Confirm cloud is set to global (or change it to this)
3. Confirm msi, cli and devicecode are selected


In [ ]:
# configure
mp.MpConfigEdit()

In [ ]:
mp.MpConfigFile().view_settings()

In [ ]:
# test
az_settings = mp.settings.settings["Azure"]

assert az_settings["cloud"] == "global"
assert all(meth in az_settings["auth_methods"] for meth in ['cli', 'msi', 'devicecode'])


Settings for Azure section should look like this:
```
{'Azure': {'auth_methods': ['cli', 'msi', 'devicecode'], 'cloud': 'global'},
```

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Finding your msticpyconfig.yaml</a>

---

MSTICPy uses the following to logic find a configuration file:
- If you specify a path using the `MSTICPYCONFIG` environment variable, it will use that
- If you have a msticpyconfig.yaml in ~/.msticpy, it will use that.
  <br>Note: - this is `$HOME/.msticpy` on Linux/Mac<br> and `%UserProfile%/.msticpy` on Windows

- If there is a msticpyconfig.yaml in the current directory, it will use that

- If you specify a path using the `config` parameter of `init_notebook`, it will use that
  and **override all of the above**.

---

You can run without a config file but you will get warnings.
<br>

You can supply
keys, connection strings, etc. to most components when you initialize them but...
#### ... who wants to do that?


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Troubleshooting settings</a>

---

1. Turn on logging before running `init_notebook`
```python
    mp.set_logging_level("INFO")
```
2. Run `mp.init_notebook` with `verbosity=2` parameter
```python
    mp.init_notebook(verbosity=2)
```
3. Comment out sections to isolate the problem section.


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Appendix - Sample msticpyconfig.yaml</a>

---

```yaml
Azure:
  auth_methods:
  - cli
  - interactive
  cloud: global
AzureSentinel:
  Workspaces:
    MainWorkspace: &id001
      ResourceGroup: <ResGroup>
      SubscriptionId: <subscription-guid>
      TenantId: <tenant-guid>
      WorkspaceId: <workspace-guid>
      WorkspaceName: <ws-name>
    Default: *id001
    OtherWorkspace:
      ResourceGroup: ResGroup
      SubscriptionId: <subscription-guid>
      TenantId: <tenant-guid>
      WorkspaceId: <workspace-guid>
      WorkspaceName: <ws-name>
DataProviders:
  Kusto-ABC:
    Args:
      Cluster: <cluster-url>
      IntegratedAuth: true
  Kusto-XYZ:
    Args:
      Cluster: <cluster-url>
      IntegratedAuth: true
  MicrosoftDefender:
    Args:
      ClientId: <client-guid>
      ClientSecret:
        KeyVault: null
      TenantId: <tenant-guid>
  MicrosoftGraph:
    Args:
      ClientId: <client-guid>
      ClientSecret:
        KeyVault: null
      TenantId: <tenant-guid>
  Mordor:
    save_folder: ~/.msticpy/mordor
    use_cached: true
KustoClusters:
  # Kusto cluster definitions (for NEW Kusto driver)
  ClusterDefaults:
    Args:
      TenantId: 69d28fd7-42a5-48bc-a619-af56397b9f28
  Cluster1:
    Args:
      Cluster: <cluster-url>
  Cluster2:
    Args:
      Cluster: <cluster-url>
      Database: EventLogs
KeyVault:
  Authority: global
  AzureRegion: East US
  ResourceGroup: <ResGroup>
  SubscriptionId: <subscription-guid>
  TenantId: <tenant-guid>
  UseKeyring: false
  VaultName: <name>
OtherProviders:
  GeoIPLite:
    Args:
      AuthKey:
        KeyVault: null
      DBFolder: ~/.msticpy
    Provider: GeoLiteLookup
  IPStack:
    Args:
      AuthKey:
        KeyVault: null
    Provider: IPStackLookup
TIProviders:
  GreyNoise:
    Args:
      AuthKey:
        KeyVault: null
    Primary: true
    Provider: GreyNoise
  OTX:
    Args:
      AuthKey:
        KeyVault: null
    Primary: true
    Provider: OTX
  OpenPageRank:
    Args:
      AuthKey:
        KeyVault: null
    Primary: true
    Provider: OPR
  RiskIQ:
    Args:
      ApiID: 
      AuthKey:
        KeyVault: null
    Primary: true
    Provider: RiskIQ
  TorExitNodes:
    Primary: true
    Provider: Tor
  VirusTotal:
    Args:
      AuthKey:
        KeyVault: null
    Primary: true
    Provider: VirusTotal
  XForce:
    Args:
      ApiID:
        KeyVault: null
      AuthKey:
        KeyVault: null
    Primary: true
    Provider: XForce
```